In [26]:
import spacy
from spacy import displacy
import pandas as pd
import spacy_annotator as spa
import random

## Dependency Parsing Using Large Japanese Model

In [3]:
##large model
nlp=spacy.load('ja_core_news_lg')
##I can't log in to the app.
doc=nlp('アプリのログインができません。')
displacy.render(doc,style='dep',jupyter=True)

## Tokanizing Text For Annotation 

In [23]:
from spacy.lang.ja.examples import sentences 

nlp = spacy.load("ja_core_news_sm")
doc = nlp("車外画像データはどのくらいの期間保存されるのですか？")
for token in doc:
    print(token.i,token.text)

0 車外
1 画像
2 データ
3 は
4 どの
5 くらい
6 の
7 期間
8 保存
9 さ
10 れる
11 の
12 です
13 か
14 ？


## DataFrame of all the Japanese text Corpus

In [7]:
df=pd.DataFrame({'text':['パンクしたときの対処方法は？','バッテリーがあがったときの対処法は？','スマートキーが正常に作動しないときのエンジンのかけ方は？','エンジンがかからない時は、どうしたらいいですか。','アプリのログインができません。','ドアが開いているときのうっかり通知は何分後に来るのですか？','セキュリティ通知履歴は削除できますか？','「次回のメンテナンス」では何が表示されていますか？','ヘルスチェックではどのような情報がわかるのですか？','走行データではどのような情報が表示されますか？','犯罪捜査等の目的で、警察等から車外画像データの提供を求められた場合は、提供するのですか？','車外画像データはどのくらいの期間保存されるのですか？']})

## Annotating the Japanese Data using the Custom tags using spacys in-built annotator 

In [10]:
import spacy
import pandas as pd
nlp = spacy.load("ja_core_news_sm")

annotator=spa.Annotator(labels=['Symptom','Vehicle','Component'],model=nlp)
df_labels=annotator.annotate(df=df,col_text="text")

HTML(value='-1 examples annotated, 13 examples left')

Text(value='', description='Symptom', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='Vehicle', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='Component', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three'…

Output()

In [24]:
annotation_data=[]
for i in df_labels.annotations:
    annotation_data.append(i)

## Annotated Data format  required By Spacy for Training 

In [25]:
annotation_data

[('パンクしたときの対処方法は？', {'entities': []}),
 ('バッテリーがあがったときの対処法は？', {'entities': [(0, 5, 'Component')]}),
 ('スマートキーが正常に作動しないときのエンジンのかけ方は？',
  {'entities': [(0, 6, 'Component'), (18, 22, 'Component')]}),
 ('エンジンがかからない時は、どうしたらいいですか。', {'entities': [(0, 4, 'Component')]}),
 ('アプリのログインができません。', {'entities': [(4, 8, 'Symptom'), (0, 3, 'Component')]}),
 ('ドアが開いているときのうっかり通知は何分後に来るのですか？',
  {'entities': [(3, 5, 'Symptom'),
    (11, 15, 'Symptom'),
    (15, 17, 'Symptom'),
    (0, 2, 'Component')]}),
 ('セキュリティ通知履歴は削除できますか？',
  {'entities': [(0, 6, 'Symptom'),
    (6, 8, 'Symptom'),
    (8, 10, 'Symptom'),
    (11, 13, 'Symptom')]}),
 ('「次回のメンテナンス」では何が表示されていますか？',
  {'entities': [(21, 23, 'Symptom'),
    (4, 10, 'Component'),
    (15, 17, 'Component')]}),
 ('ヘルスチェックではどのような情報がわかるのですか？',
  {'entities': [(0, 3, 'Symptom'),
    (3, 7, 'Component'),
    (14, 16, 'Component')]}),
 ('走行データではどのような情報が表示されますか？',
  {'entities': [(0, 2, 'Symptom'),
    (19, 21, 'Symptom'),
    (2, 5, 'Component'),
    (12, 14, '

## Training The custom NER model Using Spacy 

In [27]:

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('ja')  # create blank Language class
    
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(annotation_data, 20)

# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)
jap_spcy=spacy.load(modelfile)


#Test your text
test_text = input("Enter your testing text: ")
doc = jap_spcy(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

/home/carlton/anaconda3/lib/python3.6/site-packages/spacy/language.py:636: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs


Statring iteration 0
{'ner': 105.58277493808419}
Statring iteration 1
{'ner': 55.17429130548612}
Statring iteration 2
{'ner': 38.366269269872646}
Statring iteration 3
{'ner': 29.092391736130047}
Statring iteration 4
{'ner': 18.502367129865835}
Statring iteration 5
{'ner': 13.46200275926064}
Statring iteration 6
{'ner': 6.5006507800672795}
Statring iteration 7
{'ner': 2.3350711407307356}
Statring iteration 8
{'ner': 0.9592627314241318}
Statring iteration 9
{'ner': 0.6416178959449673}
Statring iteration 10
{'ner': 2.486844201011565}
Statring iteration 11
{'ner': 0.13356162648160116}
Statring iteration 12
{'ner': 1.8734043890197458}
Statring iteration 13
{'ner': 0.05483861710958841}
Statring iteration 14
{'ner': 0.00013765672878710822}
Statring iteration 15
{'ner': 0.0012667909886846854}
Statring iteration 16
{'ner': 0.0018168450736335744}
Statring iteration 17
{'ner': 0.0023903971112084208}
Statring iteration 18
{'ner': 2.2609548201231832e-05}
Statring iteration 19
{'ner': 1.117082603980

## Testing the Trained model by loading the saved file 

In [28]:
jap_spcy=spacy.load('Japanese_poc')


#Test your text
test_text = input("Enter your testing text: ")
doc = jap_spcy(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Enter your testing text: 「次回のメンテナンス」では何が表示されていますか？
メンテナンス 4 10 Component
表示 15 17 Component
ます 21 23 Symptom
